In [2]:
import os
import glob
import re
import pandas as pd
import numpy as np
import PyPDF2

path = os.getcwd()
pdf_files = glob.glob(path + "/Protocolos/*.pdf")
print(len(pdf_files))

196


In [3]:
pdf_files_revisar = glob.glob(path + "/Protocolos/Revisar/*.pdf")
print(len(pdf_files_revisar))

357


In [4]:
def convert_pdf_to_txt(path):
    pdf_file = open(path, 'rb')
    read_pdf = PyPDF2.PdfReader(pdf_file)
    number_of_pages = len(read_pdf.pages)
    page_content = ""
    for page_number in range(number_of_pages):
        page = read_pdf.pages[page_number]
        page_content += page.extract_text()
    pdf_file.close()
    return page_content

text = []
text_revisar = []
index=1

print("Proceso 1")
for i in pdf_files:
    print(f"{index}/{len(pdf_files)}", end='\r')
    text.append(convert_pdf_to_txt(i))
    index+=1
    
# print("Proceso 2")
# index=1
# for i in pdf_files_revisar:
#     print(f"{index}/{len(pdf_files_revisar)}", end='\r')
#     text_revisar.append(convert_pdf_to_txt(i))
#     index+=1

Proceso 1


In [62]:
def get_title(documento):
    if re.search(r"Trabajo\s*Terminal\s*No\.?", documento, flags = re.IGNORECASE):
        documento = re.split(r'Trabajo\s*Terminal\s*No\.?', documento, maxsplit=1, flags = re.IGNORECASE | re.MULTILINE)
    else: 
        documento = re.split(r'Trabajo\s*Terminal', documento, maxsplit=1, flags = re.IGNORECASE | re.MULTILINE)
    documento = documento[0]
    documento = re.sub(r'\n', '', documento)
    documento = re.sub(r'^[^a-zA-Z]+', '', documento)
    documento = re.sub(r'\s{2,}', ' ', documento)
    documento = re.sub(r'^\s+', '', documento)
    documento = re.sub(r'\s+$', '', documento)
    return documento

def get_ttnum(documento):
    if re.search(r"Trabajo\s*Terminal\s*No\.?", documento, flags = re.IGNORECASE):
        documento = re.split(r'Trabajo\s*Terminal\s*No\.?', documento, maxsplit=1, flags = re.IGNORECASE | re.MULTILINE)
    else: 
        documento = re.split(r'Trabajo\s*Terminal', documento, maxsplit=1, flags = re.IGNORECASE | re.MULTILINE)
    documento = documento[1]
    if re.search(r'Alumno+[s]?', documento, flags = re.IGNORECASE):
        documento = re.split('Alumno+[s]?', documento, flags = re.IGNORECASE)
    elif re.search(r'Integrante+[s]?', documento, flags = re.IGNORECASE):
        documento = re.split('Integrante+[s]?', documento, flags = re.IGNORECASE)
    # abstract = get_abstract(documento[1])
    documento = documento[0]
    documento = re.sub(r' ', '', documento)
    documento = re.sub(r'\n', '', documento)
    documento = re.sub(r'No\W*', '', documento)
    documento = documento.replace('_', '')
    if re.search(r'^\d{4}[a-zA-Z]\d{3}$', documento):
        documento = documento[:4] + '-' + documento[4:]
    if not re.match(r'^\d{4}-[a-zA-Z]\d{3}$', documento):
        documento = '-'
    return documento.upper()

def get_abstract(documento):
    documento = re.split(r"Resumen", documento, maxsplit=1, flags = re.IGNORECASE | re.MULTILINE)
    documento = documento[1]
    documento = re.split(r"Palabras\s*clave", documento, maxsplit=1, flags = re.IGNORECASE | re.MULTILINE)
    documento = documento[0]
    documento = re.sub(r'\n', '', documento)
    documento = re.sub(r'\s{2,}', ' ', documento)
    documento = re.sub(r'^[^a-zA-Z]+', '', documento)
    documento = re.sub(r'\s+$', '', documento)
    return documento

def get_director(documento):
    stopword_directores = ['\s*M\.*\s*[e]?[n]?\s*C\.*\s+', '\s*Dr[a]*\.?\s+','\s*M\.*\s*[e]?[n]?\s*(ing)\.*\s+', '\s*Dr[a]*\.?\s+[e]?[n]?\s*C\.*\s+']
    stopwords = [re.compile(i, flags = re.IGNORECASE | re.MULTILINE) for i in stopword_directores]
    documento = re.split(r"director[es]?", documento, maxsplit=1, flags = re.IGNORECASE | re.MULTILINE)
    documento = documento[1]
    documento = re.split(r'Resumen', documento, maxsplit = 1, flags = re.IGNORECASE | re.MULTILINE)
    documento = documento[0]
    documento = re.sub(r'\n', '', documento)
    documento = re.split(r":", documento, maxsplit=1, flags = re.IGNORECASE | re.MULTILINE)
    documento = documento[1]
    if re.search(r"e\s*[-]*\s*mail", documento, flags = re.IGNORECASE):
        documento = re.split(r"e\s*[-]*\s*mail", documento, maxsplit=1, flags = re.IGNORECASE | re.MULTILINE)
        documento = documento[0]
    documento = re.sub(r'\*', '', documento)
    documento = re.sub(r'\s{2,}', ' ', documento)
    documento = re.sub(r'^[^a-zA-Z]+', '', documento)
    documento = re.sub(r'\s+$', '', documento)
    documento = re.sub(r'\w+@[^@]+\.[a-zA-Z]{2,}', '', documento)
    if re.search(r"turno", documento, flags = re.IGNORECASE):
        documento = re.split(r"turno", documento, maxsplit=1, flags = re.IGNORECASE | re.MULTILINE)
        documento = documento[0]
    for stopword in stopwords:
        documento = re.sub(stopword, '', documento)
    documento = documento.split(',')
    documento = [i.strip() for i in documento]
    return documento

def get_keywords(documento):
    documento = re.split(r"Palabra[s]?\s*Clave[s]?\.?", documento, maxsplit=1, flags = re.IGNORECASE | re.MULTILINE)
    documento = documento[1]
    documento = re.split(r"\.", documento, maxsplit=1, flags = re.IGNORECASE | re.MULTILINE)
    documento = documento[0]
    documento = re.sub(r'^[^a-zA-Z]+', '', documento)
    documento = re.sub(' +', ' ', documento)
    documento = re.sub(r'\n', '', documento)
    documento = re.split(r"\,", documento, flags = re.IGNORECASE | re.MULTILINE)
    documento = [i.strip() for i in documento]
    documento = [i for i in documento if len(i) < 30]
    return documento

def get_areas(documento):
    directores = get_director(documento)
    areas = [''] * len(directores)
    for j in range(len(directores)):
        try:
            areas[j] = re.split(directores[j], documento, flags = re.IGNORECASE | re.MULTILINE)
            areas[j] = areas[j][2]
        except:
            try:
                directores[j] = re.sub(r'á', 'a', directores[j], flags = re.IGNORECASE)
                directores[j] = re.sub(r'é', 'e', directores[j], flags = re.IGNORECASE)
                directores[j] = re.sub(r'í', 'i', directores[j], flags = re.IGNORECASE)
                directores[j] = re.sub(r'ó', 'o', directores[j], flags = re.IGNORECASE)
                directores[j] = re.sub(r'ú', 'u', directores[j], flags = re.IGNORECASE)
                areas[j] = re.split(directores[j], documento, flags = re.IGNORECASE | re.MULTILINE)
                areas[j] = areas[j][1]
            except:
                areas[j] = '-'
        finally:
            if not areas[j] == '-':
                areas[j] = re.split(r"Firma", areas[j], flags = re.IGNORECASE | re.MULTILINE)    
                areas[j] = areas[j][0]
                if re.search(r'[Á]?[A]?rea[s]?\s*(de)?\s*inter[e]?[é]?s', areas[j], flags = re.IGNORECASE):
                    areas[j] = re.split(r'[Á]?[A]?rea[s]?\s*de\s*inter[e]?[é]?s', areas[j], flags = re.IGNORECASE | re.MULTILINE)
                    areas[j] = areas[j][1]
                    areas[j] = re.split(r"\.", areas[j], maxsplit=1, flags = re.IGNORECASE | re.MULTILINE)
                    areas[j] = areas[j][0]
                    areas[j] = re.sub(r'^[^a-zA-Z]+', '', areas[j])
                    areas[j] = re.sub(r'\s{2,}', ' ', areas[j])
                    areas[j] = re.sub(r'\n', '', areas[j])
                    areas[j] = re.split(r"\,", areas[j], flags = re.IGNORECASE | re.MULTILINE)
                    areas[j] = [i.strip() for i in areas[j]]
                else:
                    areas[j] = '-'
    return areas


def get_objetivo(documento):
    objetivoGeneral = ''
    i=1
    while len(objetivoGeneral)<50:
        objetivo = re.split(r"[1-9]\.?\s*[-]?\s*O\s*b\s*j\s*e\s*t\s*i\s*v\s*o", documento, maxsplit=i, flags = re.IGNORECASE | re.MULTILINE)
        objetivo = objetivo[i]
        objetivo = re.split(r"[2-9]\.?\s*[-]?\s*J\s*u\s*s\s*t\s*i\s*f\s*i\s*c\s*a\s*c\s*i\s*[o]?\s*[ó]?\s*n", objetivo, maxsplit=1, flags = re.IGNORECASE | re.MULTILINE)
        objetivo = objetivo[0]
        objetivo = objetivo.replace('\n', '')
        objetivo = re.sub(' +', ' ', objetivo)
        if re.search(r"espec[i]?[í]?fico[s]?", objetivo, flags = re.IGNORECASE | re.MULTILINE):
            objetivoGeneral = re.split(r"espec[i]?[í]?fico[s]?", objetivo, maxsplit=1, flags = re.IGNORECASE | re.MULTILINE)
        elif re.search(r"particular[e]?[s]?", objetivo, flags = re.IGNORECASE | re.MULTILINE):
            objetivoGeneral = re.split(r"particular[e]?[s]?", objetivo, maxsplit=1, flags = re.IGNORECASE | re.MULTILINE)
        elif re.search(r"secundario[s]?", objetivo, flags = re.IGNORECASE | re.MULTILINE):
            objetivoGeneral = re.split(r"secundario[s]?", objetivo, maxsplit=1, flags = re.IGNORECASE | re.MULTILINE)
        try:
            objetivosEspecificos = objetivoGeneral[1]
            objetivoGeneral = objetivoGeneral[0]
        except:
            objetivosEspecificos = '-'
            objetivoGeneral = objetivo
        if len(objetivoGeneral) < 50:
            i+=1

    objetivoGeneral = re.sub(r'', '', objetivoGeneral)
    objetivoGeneral = re.sub(r'•', '', objetivoGeneral)
    objetivoGeneral = re.sub(r'◦', '', objetivoGeneral)
    objetivoGeneral = re.sub(r'○', '', objetivoGeneral)
    objetivoGeneral = re.sub(r'◘', '', objetivoGeneral)
    objetivoGeneral = re.sub(r'◙', '', objetivoGeneral)
    objetivoGeneral = re.sub(r'◈', '', objetivoGeneral)
    objetivoGeneral = re.sub(r'◇', '', objetivoGeneral)
    objetivoGeneral = re.sub(r'◆', '', objetivoGeneral)
    objetivoGeneral = re.sub(r'◄', '', objetivoGeneral)
    objetivoGeneral = re.sub(r'►', '', objetivoGeneral)
    objetivoGeneral = re.sub(r'▲', '', objetivoGeneral)
    objetivoGeneral = re.sub(r'▼', '', objetivoGeneral)
    objetivoGeneral = re.sub(r'▶', '', objetivoGeneral)
    objetivoGeneral = re.sub(r'◀', '', objetivoGeneral)

    objetivosEspecificos = re.sub(r'', '', objetivosEspecificos)
    objetivosEspecificos = re.sub(r'•', '', objetivosEspecificos)
    objetivosEspecificos = re.sub(r'◦', '', objetivosEspecificos)
    objetivosEspecificos = re.sub(r'○', '', objetivosEspecificos)
    objetivosEspecificos = re.sub(r'◘', '', objetivosEspecificos)
    objetivosEspecificos = re.sub(r'◙', '', objetivosEspecificos)
    objetivosEspecificos = re.sub(r'◈', '', objetivosEspecificos)
    objetivosEspecificos = re.sub(r'◇', '', objetivosEspecificos)
    objetivosEspecificos = re.sub(r'◆', '', objetivosEspecificos)
    objetivosEspecificos = re.sub(r'◄', '', objetivosEspecificos)
    objetivosEspecificos = re.sub(r'►', '', objetivosEspecificos)
    objetivosEspecificos = re.sub(r'▲', '', objetivosEspecificos)
    objetivosEspecificos = re.sub(r'▼', '', objetivosEspecificos)
    objetivosEspecificos = re.sub(r'▶', '', objetivosEspecificos)
    objetivosEspecificos = re.sub(r'◀', '', objetivosEspecificos)
    objetivosEspecificos = re.sub(r'●', '', objetivosEspecificos)
    
    objetivoGeneral = re.sub(r' +', ' ', objetivoGeneral)
    objetivosEspecificos = re.sub(r' +', ' ', objetivosEspecificos)
    return objetivoGeneral, objetivosEspecificos

In [58]:
def get_all_info(documento):
    titulo = get_title(documento)
    ntt= get_ttnum(documento)
    abstract = get_abstract(documento)
    keywords = get_keywords(documento)
    directores = get_director(documento)
    areas = get_areas(documento)
    objetivosG, objetivosE = get_objetivo(documento)

    return titulo, ntt, directores, areas, keywords, abstract, objetivosG, objetivosE

In [34]:
text[12]

' \n1 \n Prototipo de aplicación de identificación y reporte de baches en la Ciudad de México.  \nTrabajo Terminal 2019‐B088  \nAlumnos: Suárez Cruz Bruno1, Aguilar Miranda Joseph Alexis2. \nDirector: Dra. en C. Lorena Chavarría Báez.  Director: Dr. en C. Miguel Santiago Suárez Castañón  \ne-mail1: bsuarezc1301@alumno.ipn.mx  \ne-mail2: jaguilarm1406@alumno.ipn.mx  \n \nResumen – Aplicación  para dispositivos móviles Android que recolectarán datos de los sensores del dispositivo para identificar a \ntravés de parámetros si un conductor cae en un bache. Cuando se identifique que el usuario ha caído en un bache se tomará la \nubicación en el momento y la información se enviará a una base de datos; En caso de que el usuario quiera dar de alta un bach e sin \nnecesidad de manejar, este podrá reportarlo con la información de la ubicación de este.  \n \nPalabras clave  – Dispositivo móvil, geolocalización, nube, sensores.  \n \n \n1. Introducción  \n \nEl parque vehicular en la Ciudad de Méx

In [59]:
protocolos = []
i=0
for i in range(len(text)):
    protocolos.append(get_all_info(text[i]))
    print(i + 1, '/', len(text), end='\r')

In [61]:
text[2]

'MONEDERO ELECTRONICO PARA CAFETERIAS ESCOLARES, POR MEDIO DE UN SISTEMA EMBEBIDO Trabajo Terminal No. _ _ _ _- _ _ _ Alumnos: Campos Ambrocio Javier, González Gómez Javier, Rodríguez Santiaguillo Marcela Nataly Directores: Dr. Amadeo José Argüelles Cruz, M. en Ing. Alfredo Rangel Guzmán Turno para la presentación de TT: Vespertino e-mail: jca_950904@hotmail.com  Resumen – En el presente trabajo se propone implementar un ecosistema digital para las cafeterías escolares; en el cual se podrá realizar cobros de artículos mediante tecnología RFID y controlar el inventario mediante una aplicación móvil con un servidor dedicado.  Palabras clave – Sistema embebido, aplicación móvil, tecnológica RFID, base de datos, electrónica.  1. Introducción   Los pagos ‘contactless’ ha avanzado exponencialmente. En el año 2017, se estima que los pagos globales a través de NFC ascienden a 1.300 millones de dólares. Sin embargo, esta forma de pago es solo una de las muchas aplicaciones que parece tener NFC.

In [992]:
protocolos[169][4]

['SDN', 'aut ómata celular', 'centro de datos']

In [60]:
import pandas as pd
df = pd.DataFrame(protocolos, columns=['Título', 'Número de TT', 'Directores', 'Áreas', 'Palabras clave', 'Resumen', 'Objetivo general', 'Objetivos específicos'])
df.to_excel('protocolos.xlsx', index=False)

In [23]:
titulos = []
num_tt = []
abstracts = []
keywords = []
directores = []
areas_interes = []
objetivos = []
objetivos_especificos=[]
index=1

for j in documento:
    print(f"{index}/{len(pdf_files)}", end="\r")
    titulos.append(get_title(j))
    num_tt.append(get_ttnum(j))
    keywords.append(get_keywords(j))
    directores.append(get_director(j))
    areas_interes.append(get_areas(j))
    abstracts.append(get_abstract(j))
    
    objetivos.append(get_objetivo(j))
    objetivos_especificos.append(get_objetivos_especificos(j))
    index+=1



In [31]:
titulos_revisar = []
num_tt_revisar = []
abstracts_revisar = []
keywords_revisar = []
directores_revisar = []
areas_interes_revisar = []
objetivos_revisar = []
objetivos_especificos_revisar=[]
index=1

for j in text_revisar:
    print(f"{index}/{len(pdf_files_revisar)}", end="\r")

    titulos_revisar.append(get_title(j))
    num_tt_revisar.append(get_ttnum(j))
    keywords_revisar.append(get_keywords(j))
    directores_revisar.append(get_director(j))
    areas_interes_revisar.append(get_areas(j))
    abstracts_revisar.append(get_abstract(j))
    
    objetivos_revisar.append(get_objetivo(j))
    objetivos_especificos_revisar.append(get_objetivos_especificos(j))
    index+=1

es: alcántara méndez alberto jesús, martínez díaz juan carlos *email: evi_sakura@hotmail.com **email: neon_sunlight@hotmail.com resumen – se pretende desarrollar un laboratorio virtual de electrónica analógica utilizando los conocimientos de electrónica y programación adquiridos en el transcurso de la carrera, esto con el propósito de satisfacer la demanda de los equipos de medi ción que se encuentran en los laboratorios de electrónica de la escuela superior de cómputo. apoyando al mismo tiempo a la optimiz ación del espacio de almacenamiento que ocupan algunos de estos equipos, de tal manera que puedan ser portátiles y que los alumno s puedan utilizarlos con mayor comodidad y eficiencia. palabras clave – electrónica analógica, instrumentación, sistemas electrónicos, programación. 1. introducción en el presente documento se describe la propuesta para desarrollar como trabajo t erminal un laboratorio virtual de electrónica analógica con el objetivo de cumplir con los requisitos de la op

IndexError: list index out of range

In [ ]:
# guardar en un archivo de excel con las columnas titulo, N_TT, abstract, keywords, directores, objetivos
df_tt = pd.DataFrame({'titulo': titulos , 'N_TT': num_tt, 'keywords':keywords, 'abstract':abstracts})
df_tt.fillna('-', inplace=True)
df_directores = pd.DataFrame({'Directores': directores, 'areas_interes':areas_interes})
df_directores.fillna('-', inplace=True)
# save the dataframe in a excel with the name "extraccion"
df_tt.to_excel('extraccion.xlsx', index=False)
df_directores.to_excel('directores.xlsx', index=False)

df_tt.head()

,titulo,N_TT,keywords,abstract
0,prototipo de aplicación móvil para temas de f ...,2019-B037,"[realidad aumentada, m -learning, física mecán...",la propuesta consiste en el desarrollo de una ...
1,framework para la implementación de interfaz g...,2019-A053,"[framework, c++]",el propósito de este trabajo terminal es progr...
2,monedero electronico para cafeterias escolares...,-,"[sistema embebido, aplicación móvil, tecnológi...",en el presente trabajo se propone implementar ...
3,herramienta para la creación de un entorno per...,-,"[pedagogía, desarrollo web]",con este trabajo terminal (tt) se pretende cre...
4,software de evaluación de datos de espectromet...,2016-B015,[análisis de señales],este trabajo tiene la finalidad de crear un so...


In [ ]:
df_directores.head()

,Directores,areas_interes
0,[ruíz ledesma elena fabiola],-
1,"[pescador rojas miriam, coronilla contreras uk...",-
2,"[amadeo josé argüelles cruz, alfredo rangel gu...","[[sistemas embebidos, inteligencia artificial,..."
3,[carreto arellano chadwick bustos farías eduardo],"[[cómputo móvil, redes], [gestión de ambientes..."
4,[tonáhtiu arturo ramírez romero miguel hesiqui...,"[[inteligencia artificial, bases de datos, des..."


In [1]:
import spacy
from spacy.lang.es.examples import sentences 
# !python -m spacy download es_core_news_sm
nlp = spacy.load("es_core_news_sm")

titulos_lema = []
tags = ['NOUN', 'VERB', 'ADJ', 'ADV']

for i in df_tt['titulo']:
    doc = nlp(i)
    # para cada token en el doc 
    titulo_aux = ''
    for token in doc:
        # lematizar el token
        if token.pos_ in tags:
            titulo_aux += token.lemma_ + ' '
    titulos_lema.append(titulo_aux)
titulos_lema   
            

NameError: name 'df_tt' is not defined